In [13]:
import networkx as nx
import json
import ipywidgets as widgets
from IPython.display import display

# Load the JSON data from file
data = []
with open('nvda_pairs.json', 'r') as file:
    for line in file:
        if line.strip():  # Ignore empty lines
            data.append(json.loads(line))

# Process the JSON data
nodes = set()
edges = []

for entry in data:
    token1 = entry["Token1"]
    token2 = entry["Token2"]
    score = entry["Score"]
    
    nodes.add(token1)
    nodes.add(token2)
    
    # Higher score means shorter distance, we'll use the inverse of the score as distance
    distance = 1.0 / score if score != 0 else float('inf')
    
    edges.append({
        "source": token1,
        "target": token2,
        "distance": distance
    })

# Convert to JSON format required by 3d-force-graph
graph_data = {
    'nodes': [{'id': node, 'type': ''} for node in nodes],
    'links': [{'source': edge['source'], 'target': edge['target'], 'distance': edge['distance']} for edge in edges]
}

# Convert the graph data to JSON string
graph_data_json = json.dumps(graph_data, ensure_ascii=False)

# Create the HTML content for 3d-force-graph
html_content = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <style> body {{ margin: 0; }} </style>
    <script src="https://unpkg.com/3d-force-graph"></script>
</head>
<body>
    <div id="3d-graph"></div>

    <script type="importmap"> 
    {{
        "imports": {{ 
            "three": "https://unpkg.com/three/build/three.module.js" 
        }}
    }}
    </script>
    <script type="module">
        import SpriteText from "https://unpkg.com/three-spritetext/dist/three-spritetext.mjs";

        const graphData = {graph_data_json};

        const Graph = ForceGraph3D()
            (document.getElementById('3d-graph'))
                .graphData(graphData)
                .nodeLabel('id')
                .linkThreeObjectExtend(true)
                .nodeThreeObject(node => {{
                    const sprite = new SpriteText(node.id);
                    sprite.material.depthWrite = false; // make sprite background transparent
                    sprite.textHeight = 8;
                    return sprite;
                }});

        // Increase repulsion force and link distance
        Graph.d3Force('charge', d3.forceManyBody().strength(-300));
        Graph.d3Force('link', d3.forceLink().distance(link => link.distance * 200));
    </script>
</body>
</html>
"""

# Display the HTML content using ipywidgets
html_widget = widgets.HTML(value=html_content)



In [14]:
# Save the HTML content to a file
with open('kv_graph.html', 'w') as f:
    f.write(html_content)